<a href="https://colab.research.google.com/github/An-u-rag/computer-vision-sandbox/blob/main/gan_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title import libraries

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from torch.utils.data import DataLoader

import numpy as np
from PIL import Image

from tqdm import tqdm

import os

In [2]:
#@title load dataset

batch_size = 100
#TODO: define transform that turns images to torch tensors and normalizes them to (-1, 1)
#Hint: use transforms.ToTensor() and transforms.Normalize()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])


mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


trainloader = DataLoader(mnist_trainset, batch_size = batch_size, num_workers = 0, shuffle=True)

In [3]:
#@title Define generator model
class Generator(nn.Module):
  def __init__(self, noise_dim, out_dim):
    super(Generator, self).__init__()
    #noise_dim: dimension of input noise vector
    #out_dim: dimenstion of output image in our case 28 * 28

    #TODO: define fully connected network with dims: noise_dim -> 256 -> 512 -> 512 -> out_dim
    self.fc1 = nn.Linear(noise_dim, 256)
    self.fc2 = nn.Linear(256, 512)
    self.fc3 = nn.Linear(512, 512)
    self.fc4 = nn.Linear(512, out_dim)

  def forward(self, x):
    out = F.leaky_relu(self.fc1(x), negative_slope = 0.2)
    out = F.leaky_relu(self.fc2(out), negative_slope = 0.2)
    out = F.leaky_relu(self.fc3(out), negative_slope = 0.2)
    out = torch.tanh(self.fc4(out))

    return out


In [4]:
G = Generator(100, 28 * 28)
print(G)

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=784, bias=True)
)


In [5]:
#@title Define discriminator model
class Discriminator(nn.Module):
  def __init__(self, image_dim):
    super(Discriminator, self).__init__()
    #image_dim: dimension of input image. in our case 28 * 28
    #TODO define linear layers with dims image_dim -> 256 -> 128 -> 64 -> 1
    self.fc1 = nn.Linear(image_dim, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 1)


  def forward(self, x):
     out = F.leaky_relu(self.fc1(x), negative_slope = 0.2)
     out = F.dropout(out, p=0.3)
     out = F.leaky_relu(self.fc2(out), negative_slope = 0.2)
     out = F.dropout(out, p=0.3)
     out = F.leaky_relu(self.fc3(out), negative_slope = 0.2)
     out = F.dropout(out, p=0.3)
     out = torch.sigmoid(self.fc4(out))
     return out

In [6]:
D = Discriminator(28 * 28)
print(D)

Discriminator(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)


In [29]:
#@title training

lr = 0.0002 #learning rate
nepochs = 10 #number of training epochs
noise_dim = 100 #dimension of input noise vector

class Trainer:
  def __init__(self):
    self.G = Generator(noise_dim = noise_dim, out_dim = 28 * 28)
    self.D = Discriminator(image_dim = 28 * 28)

    #TODO: define optimizers. one for generator and one for discriminator
    #Hint: use torch.optim.Adam()

    self.G_optimizer = torch.optim.Adam(self.G.parameters(), lr=lr)
    self.D_optimizer = torch.optim.Adam(self.D.parameters(), lr=lr)


    #define loss function
    self.criterion = nn.BCELoss()

    self.eval_freq = 1
    self.fig_dir = './figs'
    os.makedirs(self.fig_dir, exist_ok = True)

  def run(self):
    for e in range(1, nepochs + 1):
      if e % self.eval_freq == 0:
        self.eval_step(e)
      self.train_step(e)

  def train_step(self, epoch):
    self.G.train()
    self.D.train()
    pbar = tqdm(trainloader)
    for i, data in enumerate(pbar):
      real_data, _ = data
      real_data = real_data
      D_loss = self.train_D(real_data)
      G_loss = self.train_G()

      

      pbar.set_description("Epoch: {}, G_loss = {:.4f}, D_loss = {:.4f}".format(epoch, G_loss, D_loss))

  def train_D(self, real_data):
    self.D_optimizer.zero_grad()
    D_loss = 0.
    #TODO: train discriminator
    #real data: a batch of real data with shape(batch_size, 1, 28, 28)
    #1. feed real data to D
    #2. generate labels for real data (shoud be all ones). Hint: use torch.ones()
    #3. compute loss for real data
    #4. generate noise. Hint: use torch.randn()
    #5. feed noise to G to get fake data
    #6. feed fake data to D
    #7. generate labels for fake data (shoud be all zeros). Hint: use torch.zeros()
    #8. compute loss for fake data
    #9. add losses and optimize D
    real_data = real_data.reshape(batch_size, -1)
    D_real = self.D(real_data)
    real_labels = torch.ones((batch_size, 1))
    real_loss = self.criterion(D_real, real_labels)

    noisy_data = torch.randn((batch_size, noise_dim))
    fake_data = self.G(noisy_data)
    D_fake = self.D(fake_data)
    fake_labels = torch.zeros((batch_size, 1))
    fake_loss = self.criterion(D_fake, fake_labels)
    D_loss = real_loss + fake_loss
    D_loss.backward()
    self.D_optimizer.step()
    return D_loss

  def train_G(self):
    self.G_optimizer.zero_grad()
    G_loss = 0.
    #TODO: train generator
    #1. generate noise. Hint: use torch.randn()
    #2. feed noise to G to get fake data
    #3. feed fake data to D
    #4. generate labels for fake data (shoud be all ones) (why?). Hint: use torch.zerooness()
    #5. compute loss for fake data
    #6. optimize generator
    noise = torch.randn((batch_size, noise_dim))
    D_fake_data = self.G(noise)
    labels = torch.ones((batch_size, 1))
    D_fake_pred = self.D(D_fake_data)
    G_loss = self.criterion(D_fake_pred, labels)
    G_loss.backward()
    self.G_optimizer.step()
    return G_loss


  def eval_step(self, epoch):
    self.G.eval()
    noise = torch.randn((1, noise_dim))
    image = self.G(noise).resize(28, 28)
    image = image.clamp(-1, 1).detach().cpu().numpy()
    image = ((image + 1) * 127.5).astype('uint8')
    Image.fromarray(image).save(os.path.join(self.fig_dir, 'fig_{}.png'.format(epoch)))

In [30]:
trainer = Trainer()
trainer.run()

Epoch: 10, G_loss = 1.0050, D_loss = 1.2161: 100%|██████████| 600/600 [00:44<00:00, 13.55it/s]


In [38]:
#@title Generate samples using trained generator

G = Generator(noise_dim, 28 * 28)
G.load_state_dict(torch.load('gen_weights.pth'))

#TODO: load weights into model from gen_weights.pth
#TODO: generate 8 fake samples and plot them
samples = torch.randn((8, 100))

gen = G(samples)
gen = gen.reshape(8, -1).clamp(-1, 1).detach().cpu().numpy()

In [39]:
gen = ((gen.reshape(8, 28, 28)+1)*0.5*255).astype('uint8')

In [40]:
gen = np.concatenate(gen, axis=0)
Image.fromarray(gen)